# Task 2

In [0]:
# We need to install 'ipython_unittest' to run unittests in a Jupyter notebook
!pip install -q ipython_unittest

You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.


In [0]:
# Loading modules that we need
import unittest
from pyspark.sql.dataframe import DataFrame
from typing import Any

In [0]:
# A helper function to load a table (stored in Parquet format) from DBFS as a Spark DataFrame 
def load_df(table_name: "name of the table to load") -> DataFrame:
    return spark.read.format("delta").load(table_name)

users_df = load_df("/user/hive/warehouse/users")
comments_df = load_df("/user/hive/warehouse/comments")
posts_df = load_df("/user/hive/warehouse/posts")

#### Subtask 1: implenenting two helper functions
Impelment these two functions:
1. 'run_query' that gets a Spark SQL query and run it on df which is a Spark DataFrame; it returns the content of the first column of the first row of the DataFrame that is the output of the query;
2. 'run_query2' that is similar to 'run_query' but instead of one DataFrame gets two; it returns the content of the first column of the first row of the DataFrame that is the output of the query.

Note that the result of a Spark SQL query is itself a Spark DataFrame.

In [0]:
def run_query(query: "a SQL query string", df: "the DataFrame that the query will be executed on") -> Any:
    df.createOrReplaceTempView("df")
    result = spark.sql(query)
    r = result.collect()
    return r[0][0]

def run_query2(query: "a SQL query string", df1: "DataFrame A", df2: "DataFrame B") -> Any:
    df1.createOrReplaceTempView("df1")
    df2.createOrReplaceTempView("df2")
    result = spark.sql(query)
    r = result.collect()
    return r[0][0]

In [0]:
# Loading 'ipython_unittest' so we can use '%%unittest_main' magic command
%load_ext ipython_unittest

#### Subtask 2: writing a few queries
Write the following queries in SQL to be executed by Spark in the next cell.

1. 'q1': find the 'Id' of the most recently created post ('df' is 'posts_df') 
2. 'q2': find the number users
3. 'q3': find the 'Id' of the user who posted most number of answers
4. 'q4': find the number of questions
5. 'q5': find the display name of the user who posted most number of comments

Note that 'q1' is already available below as an example. Moreover, remmebr that Spark supports ANSI SQL 2003 so your queries have to comply with that standard.

In [0]:
q1 = "SELECT Id FROM df ORDER BY CreationDate DESC limit 1"

q2 = "SELECT COUNT(*) as count FROM df"

q3 = "SELECT OwnerUserId FROM df WHERE PostTypeId = 2 GROUP BY OwnerUserId ORDER BY COUNT(PostTypeId) DESC LIMIT 1"

q4 = "SELECT COUNT(Id) as count FROM df WHERE PostTypeId = 1 ORDER BY count DESC LIMIT 1"

q5 = "SELECT df1.DisplayName FROM df1 INNER JOIN df2 ON df1.Id = df2.UserId GROUP BY df1.DisplayName ORDER BY COUNT(df2.PostId) DESC LIMIT 1"

#### Subtask 3: validating the implementations by running the tests

Run the cell below and make sure that all the tests run successfully.

In [0]:
%%unittest_main
class TestTask2(unittest.TestCase):
    def test_q1(self):
        # find the id of the most recent post
        r = run_query(q1, posts_df)
        self.assertEqual(r, 95045)

    def test_q2(self):
        # find the number of the users
        r = run_query(q2, users_df)
        self.assertEqual(r, 91616)

    def test_q3(self):
        # find the user id of the user who posted most number of answers
        r = run_query(q3, posts_df)
        self.assertEqual(r, 64377)

    def test_q4(self):
        # find the number of questions
        r = run_query(q4, posts_df)
        self.assertEqual(r, 28950)

    def test_q5(self):
        # find the display name of the user who posted most number of comments
        r = run_query2(q5, users_df, comments_df)
        self.assertEqual(r, "Neil Slater")

Success

.....
----------------------------------------------------------------------
Ran 5 tests in 32.222s

OK
Out[10]: <unittest.runner.TextTestResult run=5 errors=0 failures=0>

#### Subtask 4: answering to questions about Spark related concepts

Please answer the following questions. Write your answer in one to two short paragraphs. Don't copy-paste; instead, write your own understanding.

1. What is the difference between 'DataFrame', 'Dataset', and 'Resilient Distributed Datasets (RDD)'? 
2. When do you suggest using RDDs instead of using DataFrames?
3. What is the main benefit of using DataSets instead of DataFrames?

Write your answers in the next cell.

1.
RDD, are resilient distributed datasets, they are collections of objects that are capable of storing data across multiple nodes in a cluster and also allows them to perform processing tasks in parallel. It is fault tolerant when performing multiple transformation. RDD are mostly used for low level transformations because it does not impose a general schema and accessing data by column or attribute is not important. 

Dataframes are also a distributed collection of data points but the main difference compared to RDD is that the data is organized in columns. The data is organized in a schema to describe the data. Dataframes are also able to build relational query for the Spark Catalyst optimizer. 

Dataset contains both the benefits of Dataframes and rdds. Datasets are type safe, it is similar code syntax as RDDs but has also access to the Spark Catalyst optimizer. It is not available to use in Python. 

2.
When we choose rdd over dataframes, we want either to use the map and actions functions. If we want to read unfiltered data, without a specific schema and the transformations a mostly low level we would use rdds.

3. 
We use dataframes when we want structure in our data and when the transformations are high level. By structure we mean that we need to create a schema. 


